In [1]:
import psycopg2
import time
import re
from datetime import timedelta
import pickle
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy import stats
import neurokit2 as nk
import pandas as pd
import numpy as np
from datetime import timedelta# Assuming you have a DataFrame named separated_data with columns 'ecg_data' and 'time_in_seconds'# Define the sampling frequency

import tensorflow as tf
from sklearn.preprocessing import StandardScaler

In [2]:
fs = 250  # Sampling frequency# Define the window size and shifting interval in seconds
window_size = 30
shifting_interval = 1# Initialize lists to store extracted features for each row

# Shooting_game Data

In [3]:
DB_NAME = "tack"

conn = psycopg2.connect(
    host="127.0.0.1",
    port="5432",
    database=DB_NAME,
    user="postgres",
    password="postgres"
)


In [4]:
# Execute the SQL query
cursor = conn.cursor()
sql_query = "SELECT * FROM tack.sensor_data"
cursor.execute(sql_query)

# Fetch all the results
columns = [desc[0] for desc in cursor.description]
data = cursor.fetchall()
ecg_df = pd.DataFrame(data, columns=columns)

cursor.close()


In [5]:
temp_df = ecg_df['sensor_data'].apply(lambda x: pd.Series(x))
ecg_df = pd.concat([ecg_df.drop('sensor_data', axis=1), temp_df.drop('eda_data',axis=1)], axis=1)
#ecg_df = ecg_df.explode(['Timestamp','ecg_data'])

# #convert to utc
# temp_time = (ecg_df['Timestamp'] * 1000000).astype('int64') #convert to microsecond
# ecg_df['utc'] = pd.to_datetime(temp_time, unit='us').dt.tz_localize('UTC').dt.tz_convert('Etc/GMT+5')

# #convert to number
# ecg_df['Timestamp'] = pd.to_numeric(ecg_df['Timestamp'], errors='raise')
# ecg_df['ecg_data'] = pd.to_numeric(ecg_df['ecg_data'], errors='raise')

ecg_df

,subject_id,computer_name,block_type,Timestamp,ecg_data
0,EDFR,mammal_panda,baseline,"[1712690031.0352373, 1712690031.0392373, 17126...","[160.0, 170.0, 157.0, 167.0, 155.0, 174.0, 165..."
1,EDFR,mammal_panda,block_01,"[1712690707.5706294, 1712690707.5746293, 17126...","[673.0, 575.0, 385.0, 125.0, 65.0, 71.0, 87.0,..."
2,EDFR,mammal_panda,block_02,"[1712691652.6205168, 1712691652.6245167, 17126...","[204.0, 210.0, 200.0, 206.0, 190.0, 205.0, 195..."
3,EDFR,mammal_panda,block_03,"[1712692657.4177406, 1712692657.4217403, 17126...","[168.0, 175.0, 165.0, 177.0, 164.0, 183.0, 177..."
4,EDFR,mammal_panda,block_04,"[1712693581.0007808, 1712693581.0047808, 17126...","[156.0, 160.0, 152.0, 203.0, 250.0, 373.0, 537..."
...,...,...,...,...,...
282,BVCX,mammal_panda,baseline,"[1713278800.638756, 1713278800.642756, 1713278...","[130.0, 129.0, 131.0, 133.0, 133.0, 137.0, 142..."
283,BVCX,mammal_panda,block_01,"[1713279427.4931395, 1713279427.4971397, 17132...","[160.0, 130.0, 128.0, 123.0, 131.0, 130.0, 132..."
284,BVCX,mammal_panda,block_02,"[1713280303.034896, 1713280303.0388958, 171328...","[490.0, 558.0, 511.0, 392.0, 277.0, 218.0, 198..."
285,BVCX,mammal_panda,block_03,"[1713281205.8859355, 1713281205.8899355, 17132...","[222.0, 224.0, 220.0, 227.0, 219.0, 225.0, 220..."


# Calculate ECG Feature

In [6]:
#calculate ECG data function

def get_ecg_features(ecg, time_in_sec, fs):
    """
    Compute ECG features from raw ECG signal.

    Parameters
    ----------
    ecg : array-like
        Raw ECG signal.
    time_in_sec : array-like
        Timestamps corresponding to each sample of the ECG signal.
    fs : float
        Sampling frequency of the ECG signal.

    Returns
    -------
    array
        Array of ECG features: [mean heart rate, maximum heart rate, minimum heart rate, heart rate variability].
    """
    try:
        b, a = butter(4, (0.25, 25), 'bandpass', fs=fs)
        ecg_filt = filtfilt(b, a, ecg, axis=0)
        ecg_cleaned = nk.ecg_clean(ecg_filt, sampling_rate=fs)
        instant_peaks, rpeaks = nk.ecg_peaks(ecg_cleaned, sampling_rate=fs,method="engzeemod2012")
    except Exception as e:
        raise ValueError("Error processing ECG signal: " + str(e))

    rr_times = time_in_sec[rpeaks['ECG_R_Peaks']]
    if len(rr_times) == 0:
        raise ValueError("No R-peaks detected in ECG signal.")
    
    # Assuming d_rr contains the time intervals between successive heartbeats in seconds
    d_rr = np.diff(rr_times)
    heart_rate = 60 / d_rr
    if heart_rate.size == 0:
        raise ValueError("Error computing heart rate from ECG signal.")
    
    valid_heart_rate = heart_rate[~np.isnan(heart_rate)]
    z_scores = np.abs(stats.zscore(valid_heart_rate))

    # Define a z-score threshold beyond which a value is considered an outlier
    z_score_threshold = 2.0

    # Remove outliers from the valid_heart_rate array
    heart_rate = valid_heart_rate[z_scores <= z_score_threshold]

    hr_mean = np.nanmean(heart_rate)
    hr_min = np.nanmin(heart_rate)
    hr_max = np.nanmax(heart_rate)
    d_rr_ms = 1000 * d_rr
    d_d_rr_ms = np.diff(d_rr_ms)

    valid_d_d_rr_ms = d_d_rr_ms[~np.isnan(d_d_rr_ms)] 
    z_scores = np.abs(stats.zscore(valid_d_d_rr_ms))
    d_d_rr_ms= valid_d_d_rr_ms[z_scores <= z_score_threshold]
    heart_rate_variability = np.sqrt(np.nanmean(np.square(d_d_rr_ms)))

    # Create a new signal 'ecg_with_rr_intervals' with RR intervals and a 1-second window around each RR interval
    ecg_with_rr_intervals = []
    ecg_with_rr_intervals_cleaned = []

    for rr_interval in rr_times:
        start_time = rr_interval - 0.1 # 1 second before the RR interval
        end_time = rr_interval + 0.1   # 1 second after the RR interval
        indices = np.where((time_in_sec >= start_time) & (time_in_sec <= end_time))[0]

        # Validate indices to ensure they are within bounds
        indices = indices[(indices >= 0) & (indices < len(ecg))]

        if len(indices) > 0:
            ecg_with_rr_intervals.extend(ecg[indices])
            ecg_with_rr_intervals_cleaned.extend(ecg_cleaned[indices])

    # Convert lists to NumPy arrays
    ecg_with_rr_intervals = np.array(ecg_with_rr_intervals)
    ecg_with_rr_intervals_cleaned = np.array(ecg_with_rr_intervals_cleaned)

    # Calculate noise power (mean squared amplitude of noise)
    signal_power = np.var(ecg_with_rr_intervals)
    noise_power = np.var(ecg_with_rr_intervals - ecg_with_rr_intervals_cleaned)

    # Calculate noise power (mean squared amplitude of noise)
    #signal_power = np.var(ecg)
    #noise_power = np.var(ecg - ecg_cleaned)

     # Calculate SNR in dB and append it to the array
    snr_values = 10 * np.log10(signal_power / noise_power)
    
    return hr_mean, hr_max, hr_min, heart_rate_variability, snr_values

In [7]:
#allias this dataFrame to work with Murat code
separated_data = ecg_df

In [8]:
# Initialize lists to store extracted features for each row
hr_mean_lists = []
hr_max_lists = []
hr_min_lists = []
hr_variability_lists = []
snr_values_lists = []
start_time_lists = []
end_time_lists = []

for index, row in separated_data.iterrows():
    print(index, ':  ',row['subject_id'], '  ', row['block_type'])
    ecg = np.array(row['ecg_data'])  # Convert ecg_data to NumPy array
    #timestamps = np.array(row['time_in_seconds'])  # Convert time_in_seconds to NumPy array    i = 0
    timestamps = np.array(row['Timestamp'])  # Convert time_in_seconds to NumPy array    i = 0
    # Initialize lists to store features for the current row
    hr_mean_list = []
    hr_max_list = []
    hr_min_list = []
    hr_variability_list = []
    snr_values_list = []    # Iterate through the ECG data in 30-second windows with 1-second overlap
    start_time_list = []
    end_time_list = []
    
    for start_time in range(0, len(ecg) - window_size * fs, shifting_interval * fs):
        end_time = start_time + window_size * fs
        window_ecg = ecg[start_time:end_time]
        window_timestamps = timestamps[start_time:end_time]        # Check for NaN values in the window_ecg and window_timestamps

        
        nan_ecg_indices = np.isnan(window_ecg)
        nan_timestamps_indices = np.isnan(window_timestamps)
        if nan_ecg_indices.any() or nan_timestamps_indices.any():
            # Interpolate NaN values if present
            if nan_ecg_indices.any():
                window_ecg = np.interp(np.arange(len(window_ecg)), np.where(~nan_ecg_indices)[0], window_ecg[~nan_ecg_indices])
            if nan_timestamps_indices.any():
                window_timestamps = np.interp(np.arange(len(window_timestamps)), np.where(~nan_timestamps_indices)[0], window_timestamps[~nan_timestamps_indices])
        try:
            # Call your get_ecg_features function and extract features
            features = get_ecg_features(window_ecg, window_timestamps, fs)            # Check if features is not None before appending
            if features is not None:
                #i = i + 1                # Append the extracted features to the respective lists for the current row
                hr_mean_list.append(features[0])
                hr_max_list.append(features[1])
                hr_min_list.append(features[2])
                hr_variability_list.append(features[3])
                snr_values_list.append(features[4])
                start_time_list.append(timestamps[start_time])
                end_time_list.append(timestamps[end_time])
                
        except ValueError as e:
            # Handle errors or skip this window in case of an error
            print(f"Error in feature extraction: {e}")
            continue    # Append the lists for the current row to the corresponding lists for all rows

    
    hr_mean_lists.append(hr_mean_list)
    hr_max_lists.append(hr_max_list)
    hr_min_lists.append(hr_min_list)
    hr_variability_lists.append(hr_variability_list)
    snr_values_lists.append(snr_values_list)# Add the lists as new columns in the separated_data DataFrame
    start_time_lists.append(start_time_list)
    end_time_lists.append(end_time_list)

separated_data['hr_mean'] = hr_mean_lists
separated_data['hr_max'] = hr_max_lists
separated_data['hr_min'] = hr_min_lists
separated_data['heart_rate_variability'] = hr_variability_lists
separated_data['snr_values'] = snr_values_lists
separated_data['start_time'] = start_time_lists
separated_data['end_time'] = end_time_lists

0 :   EDFR    baseline
1 :   EDFR    block_01
2 :   EDFR    block_02
3 :   EDFR    block_03
4 :   EDFR    block_04
5 :   HGFD    baseline
6 :   HGFD    block_01
7 :   HGFD    block_02
8 :   HGFD    block_03
9 :   HGFD    block_04
10 :   JHGF    baseline
11 :   JHGF    block_01
12 :   JHGF    block_02
13 :   JHGF    block_03
14 :   JHGF    block_04
15 :   LKMN    baseline
16 :   LKMN    block_01
17 :   LKMN    block_02
18 :   LKMN    block_03
19 :   LKMN    block_04
20 :   LPDW    baseline
21 :   LPDW    block_01
22 :   LPDW    block_02
23 :   LPDW    block_03
24 :   LPDW    block_04
25 :   MJUY    baseline
26 :   MJUY    block_01
27 :   MJUY    block_02
28 :   MJUY    block_03
29 :   MJUY    block_04
30 :   SPKZR    baseline
31 :   SPKZR    block_01
32 :   SPKZR    block_02
33 :   SPKZR    block_03
34 :   SVTZM    baseline
35 :   SVTZM    block_01
36 :   SVTZM    block_02
37 :   SVTZM    block_03
38 :   TYDF    baseline
39 :   TYDF    block_01
40 :   TYDF    block_02
41 :   TYDF    blo

C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:57: RuntimeWarning: Mean of empty slice
  heart_rate_variability = np.sqrt(np.nanmean(np.square(d_d_rr_ms)))
C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:57: RuntimeWarning: Mean of empty slice
  heart_rate_variability = np.sqrt(np.nanmean(np.square(d_d_rr_ms)))
C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:57: RuntimeWarning: Mean of empty slice
  heart_rate_variability = np.sqrt(np.nanmean(np.square(d_d_rr_ms)))
C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:57: RuntimeWarning: Mean of empty slice
  heart_rate_variability = np.sqrt(np.nanmean(np.square(d_d_rr_ms)))


Error in feature extraction: Error computing heart rate from ECG signal.
Error in feature extraction: Error computing heart rate from ECG signal.
Error in feature extraction: Error computing heart rate from ECG signal.
Error in feature extraction: Error computing heart rate from ECG signal.


C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:57: RuntimeWarning: Mean of empty slice
  heart_rate_variability = np.sqrt(np.nanmean(np.square(d_d_rr_ms)))
C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:57: RuntimeWarning: Mean of empty slice
  heart_rate_variability = np.sqrt(np.nanmean(np.square(d_d_rr_ms)))
C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:57: RuntimeWarning: Mean of empty slice
  heart_rate_variability = np.sqrt(np.nanmean(np.square(d_d_rr_ms)))
C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:57: RuntimeWarning: Mean of empty slice
  heart_rate_variability = np.sqrt(np.nanmean(np.square(d_d_rr_ms)))
C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:57: RuntimeWarning: Mean of empty slice
  heart_rate_variability = np.sqrt(np.nanmean(np.square(d_d_rr_ms)))
C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:48: RuntimeWarning: Mean of empty slice
  hr_mean = np.nanmean(heart_ra

Error in feature extraction: zero-size array to reduction operation fmin which has no identity
Error in feature extraction: zero-size array to reduction operation fmin which has no identity
Error in feature extraction: zero-size array to reduction operation fmin which has no identity


...

Error in feature extraction: zero-size array to reduction operation fmin which has no identity
Error in feature extraction: zero-size array to reduction operation fmin which has no identity
Error in feature extraction: zero-size array to reduction operation fmin which has no identity
272 :   QAZW    baseline
273 :   QAZW    block_01
274 :   QAZW    block_02
275 :   XCVH    baseline
276 :   QAZW    block_03
277 :   XCVH    block_01
278 :   QAZW    block_04
279 :   XCVH    block_02
280 :   XCVH    block_03
281 :   XCVH    block_04
282 :   BVCX    baseline
283 :   BVCX    block_01
Error in feature extraction: No R-peaks detected in ECG signal.
Error in feature extraction: zero-size array to reduction operation fmin which has no identity
Error in feature extraction: zero-size array to reduction operation fmin which has no identity
Error in feature extraction: zero-size array to reduction operation fmin which has no identity


C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:48: RuntimeWarning: Mean of empty slice
  hr_mean = np.nanmean(heart_rate)
C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:48: RuntimeWarning: Mean of empty slice
  hr_mean = np.nanmean(heart_rate)
C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:48: RuntimeWarning: Mean of empty slice
  hr_mean = np.nanmean(heart_rate)
C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:57: RuntimeWarning: Mean of empty slice
  heart_rate_variability = np.sqrt(np.nanmean(np.square(d_d_rr_ms)))
C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:57: RuntimeWarning: Mean of empty slice
  heart_rate_variability = np.sqrt(np.nanmean(np.square(d_d_rr_ms)))
C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:57: RuntimeWarning: Mean of empty slice
  heart_rate_variability = np.sqrt(np.nanmean(np.square(d_d_rr_ms)))


Error in feature extraction: Error computing heart rate from ECG signal.
284 :   BVCX    block_02


C:\Users\brook\AppData\Local\Temp\ipykernel_43848\2485472108.py:48: RuntimeWarning: Mean of empty slice
  hr_mean = np.nanmean(heart_rate)


Error in feature extraction: zero-size array to reduction operation fmin which has no identity
Error in feature extraction: No R-peaks detected in ECG signal.
Error in feature extraction: Error computing heart rate from ECG signal.
285 :   BVCX    block_03
286 :   BVCX    block_04


In [9]:
ecg_df

,subject_id,computer_name,block_type,Timestamp,ecg_data,hr_mean,hr_max,hr_min,heart_rate_variability,snr_values,start_time,end_time
0,EDFR,mammal_panda,baseline,"[1712690031.0352373, 1712690031.0392373, 17126...","[160.0, 170.0, 157.0, 167.0, 155.0, 174.0, 165...","[90.71884287767276, 90.53768858069103, 90.2537...","[102.04088881536292, 102.04088881536292, 102.0...","[84.26964734483123, 84.26964734483123, 84.2696...","[16.399785026557797, 15.889890099634755, 14.00...","[6.4596347987266025, 6.469030541974598, 6.4558...","[1712690031.0352373, 1712690032.0352373, 17126...","[1712690061.0352373, 1712690062.0352373, 17126..."
1,EDFR,mammal_panda,block_01,"[1712690707.5706294, 1712690707.5746293, 17126...","[673.0, 575.0, 385.0, 125.0, 65.0, 71.0, 87.0,...","[94.28375271973553, 94.15796232666496, 94.1161...","[103.44820442159282, 102.73970656557286, 103.4...","[83.33333002196431, 83.33333002196431, 83.3333...","[19.585919745480158, 19.49605824648644, 19.714...","[6.498243535103615, 6.647450638921232, 6.66762...","[1712690707.5706294, 1712690708.5706294, 17126...","[1712690737.5706294, 1712690738.5706294, 17126..."
2,EDFR,mammal_panda,block_02,"[1712691652.6205168, 1712691652.6245167, 17126...","[204.0, 210.0, 200.0, 206.0, 190.0, 205.0, 195...","[90.57860623428945, 91.15959687906414, 91.2140...","[96.77421141117675, 99.99997615814777, 99.9999...","[81.52173236998414, 81.52173236998414, 81.5217...","[21.71636792698553, 21.954507870312174, 22.671...","[6.512274890649863, 6.510433503317268, 6.52492...","[1712691652.6205168, 1712691653.6205168, 17126...","[1712691682.6205168, 1712691683.6205168, 17126..."
3,EDFR,mammal_panda,block_03,"[1712692657.4177406, 1712692657.4217403, 17126...","[168.0, 175.0, 165.0, 177.0, 164.0, 183.0, 177...","[91.15438169816801, 90.8287098673794, 90.72851...","[102.73970656557286, 102.73970656557286, 102.7...","[77.72017576259942, 76.14214469650187, 76.1421...","[25.90788511639349, 24.5576260170856, 24.69091...","[6.529359477225646, 6.6211698307016755, 6.6304...","[1712692657.4177406, 1712692658.4177406, 17126...","[1712692687.4177406, 1712692688.4177406, 17126..."
4,EDFR,mammal_panda,block_04,"[1712693581.0007808, 1712693581.0047808, 17126...","[156.0, 160.0, 152.0, 203.0, 250.0, 373.0, 537...","[94.19660680370845, 94.37977663535663, 94.4378...","[104.16667115081259, 104.16667115081259, 104.1...","[86.20688426697944, 86.20688426697944, 86.2068...","[13.053778904156031, 13.053775251172436, 13.11...","[6.401054542140584, 6.617534320809227, 6.62687...","[1712693581.0007808, 1712693582.0007808, 17126...","[1712693611.0007808, 1712693612.0007808, 17126..."
...,...,...,...,...,...,...,...,...,...,...,...,...
282,BVCX,mammal_panda,baseline,"[1713278800.638756, 1713278800.642756, 1713278...","[130.0, 129.0, 131.0, 133.0, 133.0, 137.0, 142...","[62.09212619958485, 62.063146546809556, 61.772...","[66.66665960241322, 66.66665960241322, 66.6666...","[58.59374039573753, 58.59374039573753, 56.8181...","[23.999999731716642, 29.67443201229561, 29.799...","[10.163249623956643, 10.17344991331301, 10.144...","[1713278800.638756, 1713278801.638756, 1713278...","[1713278830.638756, 1713278831.638756, 1713278..."
283,BVCX,mammal_panda,block_01,"[1713279427.4931395, 1713279427.4971397, 17132...","[160.0, 130.0, 128.0, 123.0, 131.0, 130.0, 132...","[74.9910090055507, 75.38666964171142, 75.21325...","[87.20930691923327, 87.20930691923327, 87.2093...","[66.96427473798575, 66.96427473798575, 66.9642...","[39.86844077247947, 40.65340159687865, 41.1424...","[10.541337242815722, 10.448608578909722, 10.56...","[1713279427.4931395, 1713279428.4931395, 17132...","[1713279457.4931395, 1713279458.4931395, 17132..."
284,BVCX,mammal_panda,block_02,"[1713280303.034896, 1713280303.0388958, 171328...","[490.0, 558.0, 511.0, 392.0, 277.0, 218.0, 198...","[68.95163278903887, 68.64900990795138, 68.7674...","[74.25741767920783, 74.25741767920783, 74.2574...","[64.65519088521401, 62.76150039092164, 62.7615...","[26.015318843286945, 27.75926623694406, 2

## Calculate Workload

In [10]:
ecg_model_path = r".\Mural_ECG_model\model"
ecg_model = tf.saved_model.load(ecg_model_path)

In [11]:
MAX_HR_CUTOFF = 250
MIN_HR_CUTOFF = 25
SNR_CUTOFF = 4
SUBJECT_LIST = ecg_df['subject_id'].unique()
SUBJECT_LIST

array(['EDFR', 'HGFD', 'JHGF', 'LKMN', 'LPDW', 'MJUY', 'SPKZR', 'SVTZM',
       'TYDF', 'WZXE', 'XZPT', 'ZXCV', 'ZXWE', 'ZYXW', 'HJKL', 'HPLX',
       'LKPW', 'LPZV', 'LRTB', 'MJKL', 'MNOP', 'OPKJ', 'QAXY', 'QXZW',
       'SBTBT', 'STWXJ', 'VFTY', 'XUYP', 'DFGH', 'DFZX', 'FTLD', 'GHJK',
       'MTYH', 'NMBV', 'PKJH', 'QWER', 'QWRT', 'RQWE', 'SNMVN', 'SXVBN',
       'TYZX', 'ZRTY', 'CVBN', 'DKLQ', 'GTYU', 'JKLZ', 'KJHG', 'MNBW',
       'QPOI', 'SBQRG', 'SRKLP', 'UYGH', 'VBNM', 'VNPL', 'WERT', 'LKJH',
       'QAZW', 'XCVH', 'BVCX'], dtype=object)

In [12]:
for cur_subject in SUBJECT_LIST:
    temp_df = ecg_df[ecg_df['subject_id'] == cur_subject]
    baseline_row = temp_df[temp_df['block_type'] == 'baseline'].iloc[0]
    
    #fit_scaler
    mean_scaler = StandardScaler().fit(np.array(baseline_row['hr_mean']).reshape(-1,1))
    max_scaler = StandardScaler().fit(np.array(baseline_row['hr_max']).reshape(-1,1))
    min_scaler = StandardScaler().fit(np.array(baseline_row['hr_min']).reshape(-1,1))
    hrv_scaler = StandardScaler().fit(np.array(baseline_row['heart_rate_variability']).reshape(-1,1))
    
    max_hr_cutoff = (MAX_HR_CUTOFF - max_scaler.mean_[0]) / max_scaler.var_[0]   #convert heart rate cuttoff value to standardlize value
    min_hr_cutoff = (MIN_HR_CUTOFF - min_scaler.mean_[0]) / min_scaler.var_[0]   #convert heart rate cuttoff value to standardlize value

    print(max_hr_cutoff, '  ',min_hr_cutoff)
    for index, row in temp_df.iterrows():
        print(index, ':  ',row['subject_id'], '  ', row['block_type'], end = '  ')
        if len(row['hr_mean']) < 2:
            print("\n Data unavailible, Continue")
            continue
            
        #tranform data
        mean_scaled = mean_scaler.transform(np.array(row['hr_mean']).reshape(-1,1))
        max_scaled = max_scaler.transform(np.array(row['hr_max']).reshape(-1,1))
        min_scaled = min_scaler.transform(np.array(row['hr_min']).reshape(-1,1))
        hrv_scaled = hrv_scaler.transform(np.array(row['heart_rate_variability']).reshape(-1,1))
    
    
        #predicting workload
        workload_list = []
        for current_data,current_snr in zip(zip(mean_scaled, max_scaled, min_scaled, hrv_scaled), row['snr_values']):
            
            current_sample = np.array(current_data).reshape(1,-1)
            #santize check
            if (current_sample[0][1] > max_hr_cutoff) or (current_sample[0][2] < min_hr_cutoff) or current_snr < (SNR_CUTOFF):
                print("\n Invalid heart_rate: ", current_sample[0][1], "  ", current_sample[0][2], "  ", current_snr)
                workload_list.append([[np.nan]])
                continue

            prediction = ecg_model(current_sample)
            workload_list.append(prediction)
    
        #add workload to dataFrame
        if len(workload_list) < 0:
            print("No workload data")
            continue
        mask = (ecg_df['subject_id'] == row['subject_id']) & (ecg_df['block_type'] == row['block_type'])
        print(ecg_df.loc[mask].shape)
        ecg_df.loc[mask,'workload'] = str(np.array(workload_list).reshape(-1).tolist())



7.15612332075501    -3.362227254754731
0 :   EDFR    baseline  (1, 12)
1 :   EDFR    block_01  
 Invalid heart_rate:  8.409009716021334    3.463838962901665    6.632505523876048

 Invalid heart_rate:  8.409009716021334    -5.342180306781803    6.5749342279731895

 Invalid heart_rate:  8.409009716021334    -5.342180306781803    6.3190106739921355

 Invalid heart_rate:  20.529820504190354    -19.125982662814344    6.371669266551909

 Invalid heart_rate:  8.409009716021334    -5.342180306781803    6.364224790149566

 Invalid heart_rate:  8.409009716021334    -5.342180306781803    6.357850043001703

 Invalid heart_rate:  8.409009716021334    -5.342180306781803    6.363625374988919

 Invalid heart_rate:  8.409009716021334    -5.342180306781803    6.361978610497036

 Invalid heart_rate:  8.409009716021334    -8.148805004019364    5.868043357730506

 Invalid heart_rate:  45.188889377786545    -8.747218801431929    5.3309192104387915

 Invalid heart_rate:  45.188889377786545    -18.27025073828

In [13]:
ecg_df

,subject_id,computer_name,block_type,Timestamp,ecg_data,hr_mean,hr_max,hr_min,heart_rate_variability,snr_values,start_time,end_time,workload
0,EDFR,mammal_panda,baseline,"[1712690031.0352373, 1712690031.0392373, 17126...","[160.0, 170.0, 157.0, 167.0, 155.0, 174.0, 165...","[90.71884287767276, 90.53768858069103, 90.2537...","[102.04088881536292, 102.04088881536292, 102.0...","[84.26964734483123, 84.26964734483123, 84.2696...","[16.399785026557797, 15.889890099634755, 14.00...","[6.4596347987266025, 6.469030541974598, 6.4558...","[1712690031.0352373, 1712690032.0352373, 17126...","[1712690061.0352373, 1712690062.0352373, 17126...","[0.3972630500793457, 0.3774046003818512, 0.337..."
1,EDFR,mammal_panda,block_01,"[1712690707.5706294, 1712690707.5746293, 17126...","[673.0, 575.0, 385.0, 125.0, 65.0, 71.0, 87.0,...","[94.28375271973553, 94.15796232666496, 94.1161...","[103.44820442159282, 102.73970656557286, 103.4...","[83.33333002196431, 83.33333002196431, 83.3333...","[19.585919745480158, 19.49605824648644, 19.714...","[6.498243535103615, 6.647450638921232, 6.66762...","[1712690707.5706294, 1712690708.5706294, 17126...","[1712690737.5706294, 1712690738.5706294, 17126...","[0.6966437697410583, 0.6849216222763062, 0.685..."
2,EDFR,mammal_panda,block_02,"[1712691652.6205168, 1712691652.6245167, 17126...","[204.0, 210.0, 200.0, 206.0, 190.0, 205.0, 195...","[90.57860623428945, 91.15959687906414, 91.2140...","[96.77421141117675, 99.99997615814777, 99.9999...","[81.52173236998414, 81.52173236998414, 81.5217...","[21.71636792698553, 21.954507870312174, 22.671...","[6.512274890649863, 6.510433503317268, 6.52492...","[1712691652.6205168, 1712691653.6205168, 17126...","[1712691682.6205168, 1712691683.6205168, 17126...","[0.3329460918903351, 0.385335236787796, 0.3966..."
3,EDFR,mammal_panda,block_03,"[1712692657.4177406, 1712692657.4217403, 17126...","[168.0, 175.0, 165.0, 177.0, 164.0, 183.0, 177...","[91.15438169816801, 90.8287098673794, 90.72851...","[102.73970656557286, 102.73970656557286, 102.7...","[77.72017576259942, 76.14214469650187, 76.1421...","[25.90788511639349, 24.5576260170856, 24.69091...","[6.529359477225646, 6.6211698307016755, 6.6304...","[1712692657.4177406, 1712692658.4177406, 17126...","[1712692687.4177406, 1712692688.4177406, 17126...","[0.3617091178894043, 0.39754578471183777, 0.39..."
4,EDFR,mammal_panda,block_04,"[1712693581.0007808, 1712693581.0047808, 17126...","[156.0, 160.0, 152.0, 203.0, 250.0, 373.0, 537...","[94.19660680370845, 94.37977663535663, 94.4378...","[104.16667115081259, 104.16667115081259, 104.1...","[86.20688426697944, 86.20688426697944, 86.2068...","[13.053778904156031, 13.053775251172436, 13.11...","[6.401054542140584, 6.617534320809227, 6.62687...","[1712693581.0007808, 1712693582.0007808, 17126...","[1712693611.0007808, 1712693612.0007808, 17126...","[0.7251088619232178, 0.7377746105194092, 0.741..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,BVCX,mammal_panda,baseline,"[1713278800.638756, 1713278800.642756, 1713278...","[130.0, 129.0, 131.0, 133.0, 133.0, 137.0, 142...","[62.09212619958485, 62.063146546809556, 61.772...","[66.66665960241322, 66.66665960241322, 66.6666...","[58.59374039573753, 58.59374039573753, 56.8181...","[23.999999731716642, 29.67443201229561, 29.799...","[10.163249623956643, 10.17344991331301, 10.144...","[1713278800.638756, 1713278801.638756, 1713278...","[1713278830.638756, 1713278831.638756, 1713278...","[0.2652081847190857, 0.2652091085910797, 0.469..."
283,BVCX,mammal_panda,block_01,"[1713279427.4931395, 1713279427.4971397, 17132...","[160.0, 130.0, 128.0, 123.0, 131.0, 130.0, 132...","[74.9910090055507, 75.38666964171142, 75.21325...","[87.20930691923327, 87.20930691923327, 87.2093...","[66.96427473798575, 66.96427473798575, 66.9642...","[39.86844077247947, 40.65340159687865, 41.1424...","[10.541337242815722, 10.448608578909722, 10.56...","[1713279427.4931395, 1713279428.4931395, 17132...","[1713279457.4931395, 1713279458.4931395, 17132...","[0.9999

In [14]:
ecg_df.drop(["Timestamp","ecg_data"],axis=1).to_csv("Shooting_game_heart_rate.csv",index=False)

conn.close()

# CO